## <img src="https://lh3.googleusercontent.com/mUTbNK32c_DTSNrhqETT5aQJYFKok2HB1G2nk2MZHvG5bSs0v_lmDm_ArW7rgd6SDGHXo0Ak2uFFU96X6Xd0GQ=w160-h128" width="45" valign="top" alt="BigQuery"> Generate Synthetic Menu data and images using Gemini Pro


### License

In [ ]:
##################################################################################
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
# 
#     https://www.apache.org/licenses/LICENSE-2.0
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
###################################################################################

### Notebook Overview

- This notebook demostrates how data engineers can increase the speed of their development work using LLMs.  In the notebook, LLMs will be used to generate synthetic data, understand an image of an ERD and create the DDL and use image generation to create realistic images of products.

- Notebook Logic:
    1. Create our the menu table by showing Gemimi Pro a picture of the tables to create.
        1. First we ened to download a picture of our ERD.
        2. We need to construct a LLM prompt telling it to read the ERD and construct our SQL.
        3. Execute the SQL to create the new table and primary keys on the table.
    2. Create a LLM prompt to generate some unique coffee names
    3. Create a LLM prompt to generate the insert statements to our menu table based upon the unique coffee name.
        - The prompt can create small, medium and large sizes.
        - The LLM is smart enough to create pricing correctly according to the size.
        - The LLM can generate our image prompt for Imagen2
        - The promp can pass in all the foreign and primary key values.
    4. Execute the generated SQL.
    5. Create a LLM prompt to create some food items.
    6. Execute the generate SQL.
    7. Read the menu items and the image prompt
    8. Geenrate the coffee and food image
    9. Upload the images to GCS

## Initialize Python

In [ ]:
project_id="${project_id}"
location="us-central1"
model_id = "imagegeneration@005"

# No need to set these
city_names=["New York City", "London", "Tokyo", "San Francisco"]
city_ids=[1,2,3,4]
city_languages=["American English", "British English", "Japanese", "American English"]
number_of_coffee_trucks = "4"

dataset_id = "data_beans_synthetic_data"

gcs_storage_bucket = "${data_beans_curated_bucket}"
gcs_storage_path = "data-beans/menu-images/"

In [ ]:
from PIL import Image
from IPython.display import HTML
import IPython.display
import google.auth
import requests
import json
import uuid
import base64
import os
import cv2

from google.cloud import bigquery
client = bigquery.Client()

## Imagen2 / Gemini Pro / Gemini Pro Vision (Helper Functions)

#### Imagen2

In [ ]:
def ImageGen(prompt):
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/image-generation
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project}/locations/{location}/publishers/google/models/imagegeneration:predict"

  payload = {
    "instances": [
      {
        "prompt": prompt
      }
    ],
    "parameters": {
      "sampleCount": 1
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    image_data = json.loads(response.content)["predictions"][0]["bytesBase64Encoded"]
    image_data = base64.b64decode(image_data)
    filename= str(uuid.uuid4()) + ".png"
    with open(filename, "wb") as f:
      f.write(image_data)
    print(f"Image generated OK.")
    return filename
  else:
    error = f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'"
    raise RuntimeError(error)

#### Gemini Pro LLM

In [ ]:
def GeminiProLLM(prompt, temperature = .8, topP = .8, topK = 40):

  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/gemini#gemini-pro
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/gemini-pro:streamGenerateContent"

  payload = {
    "contents": {
      "role": "user",
      "parts": {
          "text": prompt
      },
    },
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    },
    "generation_config": {
      "temperature": temperature,
      "topP": topP,
      "topK": topK,
      "maxOutputTokens": 8192,
      "candidateCount": 1
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    json_response = json.loads(response.content)
    llm_response = ""
    for item in json_response:
      try:
        llm_response = llm_response + item["candidates"][0]["content"]["parts"][0]["text"]
      except Exception as err:
        print(f"response.content: {response.content}")
        raise RuntimeError(err)

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")

    # print(f"llm_response:\n{llm_response}")
    return llm_response
  else:
    error = f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'"
    raise RuntimeError(error)


#### Gemini Pro Vision LLM

In [ ]:
# Use the Gemini with Vision
def GeminiProVisionLLM(prompt, imageBase64, temperature = .4, topP = 1, topK = 32):

  if temperature < 0:
    temperature = 0

  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/gemini#gemini-pro
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/gemini-pro-vision:streamGenerateContent"

  payload = {
  "contents": [
      {
        "role": "user",
        "parts": [
          {
            "text": prompt
          },
          {
            "inlineData": {
              "mimeType": "image/png",
              "data": f"{imageBase64}"
            }
          }
        ]
      }
    ],
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    },
    "generation_config": {
      "temperature": temperature,
      "topP": topP,
      "topK": topK,
      "maxOutputTokens": 2048,
      "candidateCount": 1
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    json_response = json.loads(response.content)
    llm_response = ""
    for item in json_response:
      llm_response = llm_response + item["candidates"][0]["content"]["parts"][0]["text"]

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")

    # print(f"llm_response:\n{llm_response}")
    return llm_response
  else:
    error = f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'"
    raise RuntimeError(error)


In [ ]:
# Use the Gemini with Vision
def GeminiProVisionMultipleFileLLM(prompt, image_prompt, temperature = .4, topP = 1, topK = 32):
  creds, project = google.auth.default()
  auth_req = google.auth.transport.requests.Request() # required to acess access token
  creds.refresh(auth_req)
  access_token=creds.token

  headers = {
      "Content-Type" : "application/json",
      "Authorization" : "Bearer " + access_token
  }

  # https://cloud.google.com/vertex-ai/docs/generative-ai/model-reference/gemini#gemini-pro
  url = f"https://{location}-aiplatform.googleapis.com/v1/projects/{project_id}/locations/{location}/publishers/google/models/gemini-pro-vision:streamGenerateContent"


  parts = []
  new_item = {
      "text": prompt
      }
  parts.append(new_item)

  for item in image_prompt:
    new_item = {
        "text": f"Image Name: {item['llm_image_filename']}:\n"
        }
    parts.append(new_item)
    new_item = {
        "inlineData": {
            "mimeType": "image/png",
            "data": item["llm_image_base64"]
            }
        }
    parts.append(new_item)

  payload = {
  "contents": [
      {
        "role": "user",
        "parts": parts
      }
    ],
    "safety_settings": {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "threshold": "BLOCK_LOW_AND_ABOVE"
    },
    "generation_config": {
      "temperature": temperature,
      "topP": topP,
      "topK": topK,
      "maxOutputTokens": 2048,
      "candidateCount": 1
    }
  }

  response = requests.post(url, json=payload, headers=headers)

  if response.status_code == 200:
    json_response = json.loads(response.content)
    llm_response = ""
    for item in json_response:
      llm_response = llm_response + item["candidates"][0]["content"]["parts"][0]["text"]

    # Remove some typically response characters (if asking for a JSON reply)
    llm_response = llm_response.replace("```json","")
    llm_response = llm_response.replace("```","")

    # print(f"llm_response:\n{llm_response}")
    return llm_response
  else:
    error = f"Error with prompt:'{prompt}'  Status:'{response.status_code}' Text:'{response.text}'"
    raise RuntimeError(error)


#### SQL Functions

In [ ]:
def RunQuery(sql):
  import time

  if (sql.startswith("SELECT") or sql.startswith("WITH")):
      df_result = client.query(sql).to_dataframe()
      return df_result
  else:
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    while query_job.state != "DONE":
      time.sleep(2)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

    if query_job.error_result == None:
      return True
    else:
      return False

In [ ]:
def GetNextPrimaryKey(fully_qualified_table_name, field_name):
  sql = f"""
  SELECT IFNULL(MAX({field_name}),0) AS result
    FROM `{fully_qualified_table_name}`
  """
  # print(sql)
  df_result = client.query(sql).to_dataframe()
  # display(df_result)
  return df_result['result'].iloc[0] + 1

In [ ]:
def GetTableSchema(dataset_name, table_name):
  import io

  dataset_ref = client.dataset(dataset_name, project=project_id)
  table_ref = dataset_ref.table(table_name)
  table = client.get_table(table_ref)

  f = io.StringIO("")
  client.schema_to_json(table.schema, f)
  return f.getvalue()

In [ ]:
def GetStartingValue(dataset_name, table_name, field_name):
  sql = f"""
  SELECT IFNULL(MAX({field_name}),0) + 1 AS result
    FROM `{project_id}.{dataset_name}.{table_name}`
  """
  #print(sql)
  df_result = client.query(sql).to_dataframe()
  #display(df_result)
  return df_result['result'].iloc[0]

In [ ]:
def GetForeignKeys(dataset_name, table_name, field_name):
  sql = f"""
  SELECT STRING_AGG(CAST({field_name} AS STRING), "," ORDER BY {field_name}) AS result
    FROM `{project_id}.{dataset_name}.{table_name}`
  """
  #print(sql)
  df_result = client.query(sql).to_dataframe()
  #display(df_result)
  return df_result['result'].iloc[0]

In [ ]:
def GetDistinctValues(dataset_name, table_name, field_name):
  sql = f"""
  SELECT STRING_AGG(DISTINCT {field_name}, "," ) AS result
    FROM `{project_id}.{dataset_name}.{table_name}`
  """
  #print(sql)
  df_result = client.query(sql).to_dataframe()
  #display(df_result)
  return df_result['result'].iloc[0]

In [ ]:
# Run DDL
# This has been done to make the code re-runnable for the demo
# We need to add some logic for the PK and FKs and the prompt will be updated to code gen it

def RunDDL(sql):
  import time

  sql = f"""CREATE SCHEMA IF NOT EXISTS `{project_id}.{dataset_id}`;

CREATE TABLE IF NOT EXISTS `{project_id}.{dataset_id}.menu`
(
    menu_id INT64 NOT NULL OPTIONS(description="Primary key. Menu table."),
    company_id INT64 NOT NULL OPTIONS(description="Foreign key: Company table."),
    item_name STRING NOT NULL OPTIONS(description="The name of the menu item."),
    item_price FLOAT64 NOT NULL OPTIONS(description="The price of the menu item."),
    item_description STRING NOT NULL OPTIONS(description="The description of the menu item."),
    item_size STRING NOT NULL OPTIONS(description="The size of the menu item."),
    llm_item_description_prompt STRING  OPTIONS(description="LLM prompt are all of the menu item descriptions."),
    llm_item_description string  OPTIONS(description="LLM summary are all of the menu item descriptions."),
    llm_item_image_prompt STRING  OPTIONS(description="LLM prompt are all of the menu item images."),
    llm_item_image_url string  OPTIONS(description="LLM summary are all of the menu item images.")
)
CLUSTER BY menu_id;

CREATE TABLE IF NOT EXISTS `{project_id}.{dataset_id}.company`
(
company_id INT64 NOT NULL OPTIONS(description="Primary key. Company table."),
company_name STRING NOT NULL OPTIONS(description="The name of the company.")
)
CLUSTER BY company_id;

ALTER TABLE `{project_id}.{dataset_id}.menu` DROP PRIMARY KEY IF EXISTS;
ALTER TABLE `{project_id}.{dataset_id}.menu` ADD PRIMARY KEY (menu_id) NOT ENFORCED;

ALTER TABLE `{project_id}.{dataset_id}.company`  DROP PRIMARY KEY IF EXISTS;
ALTER TABLE `{project_id}.{dataset_id}.company` ADD PRIMARY KEY (company_id) NOT ENFORCED;

ALTER TABLE `{project_id}.{dataset_id}.menu` DROP CONSTRAINT IF EXISTS `menu.fk$1`;
ALTER TABLE `{project_id}.{dataset_id}.menu` ADD FOREIGN KEY (company_id) REFERENCES `{project_id}.{dataset_id}.company`(company_id) NOT ENFORCED;
  """

  # To see the contraints in BigQuery
  # SELECT * FROM  `data-beans-demo-5r3n4jbe01.data_beans_synthetic_data.INFORMATION_SCHEMA.TABLE_CONSTRAINTS`


  job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
  query_job = client.query(sql, job_config=job_config)

  # Check on the progress by getting the job's updated state.
  query_job = client.get_job(
      query_job.job_id, location=query_job.location
  )
  print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

  while query_job.state != "DONE":
    time.sleep(2)
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
        )
    print("Job {} is currently in state {} with error result of {}".format(query_job.job_id, query_job.state, query_job.error_result))

  if query_job.error_result == None:
    return True
  else:
    return False

#### Helper Functions

In [ ]:
def convert_png_to_base64(image_path):
  image = cv2.imread(image_path)

  # Convert the image to a base64 string.
  _, buffer = cv2.imencode('.png', image)
  base64_string = base64.b64encode(buffer).decode('utf-8')

  return base64_string

In [ ]:
# This was generated by GenAI

def copy_file_to_gcs(local_file_path, bucket_name, destination_blob_name):
  """Copies a file from a local drive to a GCS bucket.

  Args:
      local_file_path: The full path to the local file.
      bucket_name: The name of the GCS bucket to upload to.
      destination_blob_name: The desired name of the uploaded file in the bucket.

  Returns:
      None
  """

  import os
  from google.cloud import storage

  # Ensure the file exists locally
  if not os.path.exists(local_file_path):
      raise FileNotFoundError(f"Local file '{local_file_path}' not found.")

  # Create a storage client
  storage_client = storage.Client()

  # Get a reference to the bucket
  bucket = storage_client.bucket(bucket_name)

  # Create a blob object with the desired destination path
  blob = bucket.blob(destination_blob_name)

  # Upload the file from the local filesystem
  content_type = ""
  if local_file_path.endswith(".html"):
    content_type = "text/html; charset=utf-8"

  if local_file_path.endswith(".json"):
    content_type = "application/json; charset=utf-8"

  if content_type == "":
    blob.upload_from_filename(local_file_path)
  else:
    blob.upload_from_filename(local_file_path, content_type = content_type)

  print(f"File '{local_file_path}' uploaded to GCS bucket '{bucket_name}' as '{destination_blob_name}.  Content-Type: {content_type}'.")

## Menu Synthetic Data and Image Generation

#### Download the ERD image and Generate our Database Schema DDL

In [ ]:
import requests
import shutil

menu_erd_filename = "Data-Beans-Menu-ERD.png"

# Specify the image URL
img_url = f"https://storage.googleapis.com/data-analytics-golden-demo/data-beans/v1/colab-supporting-images/{menu_erd_filename}"

# Send a GET request to fetch the image
response = requests.get(img_url, stream=True)

# Check for successful download
if response.status_code == 200:
    # Set decode_content to True to prevent encoding errors
    response.raw.decode_content = True

    # Open a local file in binary write mode
    with open(menu_erd_filename, "wb") as f:
        # Copy image data to the local file in chunks
        shutil.copyfileobj(response.raw, f)

    print("Image downloaded successfully!")
else:
    print("Image download failed with status code:", response.status_code)

In [ ]:
print(f"Filename: {menu_erd_filename}")
img = Image.open(menu_erd_filename)
img.thumbnail([800,389]) # width, height
IPython.display.display(img)

In [ ]:
llm_erd_prompt=f"""Use BigQuery SQL commands to create the following:
- Create a new BigQuery schema named "{dataset_id}".
- Use only BigQuery datatypes.  Double and triple check this since it causes a lot of errors.
- Create the BigQuery DDLs for the attached ERD.
- Create primary keys  for each table using the ALTER command. Use the "NOT ENFORCED" keyword.
- Create foreign keys  for each table using the ALTER command. Use the "NOT ENFORCED" keyword.
- For each field add an OPTIONS for the description.
- Cluster the table by the primary key.
- For columns that can be null do not add "NULL" to the created SQL statement.  BigQuery leaves this blank.
- All ALTER TABLE statements should by at the bottom of the generated script.
- The ALTER TABLES statements should be order by the primary key statements and then the foreign key statements. Order matters!
- Double check your work especially that you used ONLY BigQuery data types.

Previous Errors that have been generated by this script.  Be sure to check your work to avoid encountering these.
- Query error: Type not found: FLOAT at [6:12]
- Query error: Table test.company does not have Primary Key constraints at [25:1]

Example:
CREATE TABLE IF NOT EXISTS `{project_id}.{dataset_id}.customer`
(
    customer_id INTEGER NOT NULL OPTIONS(description="Primary key. Customer table."),
    country_id INTEGER NOT NULL OPTIONS(description="Foreign key: Country table."),
    customer_llm_summary STRING NOT NULL OPTIONS(description="LLM generated summary of customer data."),
    customer_lifetime_value STRING NOT NULL OPTIONS(description="Total sales for this customer."),
    customer_cluster_id FLOAT NOT NULL OPTIONS(description="Clustering algorithm id."),
    customer_review_llm_summary STRING  OPTIONS(description="LLM summary are all of the customer reviews."),
    customer_survey_llm_summary STRING  OPTIONS(description="LLM summary are all of the customer surveys.")
)
CLUSTER BY customer_id;

CREATE TABLE IF NOT EXISTS `{project_id}.{dataset_id}.country`
(
country_id INTEGER NOT NULL OPTIONS(description="Primary key. Country table."),
country_name STRING NOT NULL OPTIONS(description="The name of the country.")
)
CLUSTER BY country_id;


ALTER TABLE `{project_id}.{dataset_id}.customer` ADD PRIMARY KEY (customer_id) NOT ENFORCED;
ALTER TABLE `{project_id}.{dataset_id}.country` ADD PRIMARY KEY (country_id) NOT ENFORCED;

ALTER TABLE `{project_id}.{dataset_id}.customer` ADD FOREIGN KEY (country_id) REFERENCES `{project_id}.{dataset_id}.country`(country_id) NOT ENFORCED;
"""

In [ ]:
imageBase64 = convert_png_to_base64(menu_erd_filename)

llm_response = GeminiProVisionLLM(llm_erd_prompt, imageBase64, temperature=.2, topP=1, topK=32)

In [ ]:
# Need to prompt this
llm_response = llm_response.replace("STRING NULL OPTIONS","STRING OPTIONS")
llm_response = llm_response.replace("JSON NULL OPTIONS","JSON OPTIONS")
llm_response = llm_response.replace("BOOLEAN NULL OPTIONS","BOOLEAN OPTIONS")

# Run this by hand (it will create a new dataset so we do not overwrite our main demo dataset)
print(llm_response)

In [ ]:
RunDDL(llm_response)

### Generate Data

#### Company Table

In [ ]:
company_count = 1

table_name = "company"
primary_key = "company_id"

schema = GetTableSchema(dataset_id, table_name)

In [ ]:
company_names_prompt = f"""Generate {company_count} creative names and return in the below json format.
- The name should be new and not a name that is already used by an existing coffee company.
- The name should be related to coffee.
- The name should be related to a food truck type of service.

JSON format: [ "value" ]
Sample JSON Response: [ "value1", "value2" ]
"""

llm_success = False
temperature=.8
while llm_success == False:
  try:
    company_names = GeminiProLLM(company_names_prompt, temperature=temperature, topP=.8, topK = 40)
    company_names_json = json.loads(company_names)
    llm_success = True
  except:
    # Reduce the temperature for more accurate generation
    temperature = temperature - .05
    print("Regenerating...")

print(f"company_names: {company_names}")

In [ ]:
company_names = ["Data Beans"]

In [ ]:
starting_value = GetStartingValue(dataset_id, table_name, primary_key)

company_names_sql_prompt=f"""
You are a database engineer and need to generate data for a table for the below schema.
- The schema is for a Google Cloud BigQuery Table.
- The table name is "{project_id}.{dataset_id}.{table_name}".
- Read the description of each field for valid values.
- Do not preface the response with any special characters or 'sql'.
- Generate {company_count} insert statements for this table.
- Valid values for company_name are: {company_names}
- The starting value of the field {primary_key} is {starting_value}.
- Only generate a single statement, not multiple INSERTs.


Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

Schema: {schema}
"""

llm_success = False
temperature=.8
while llm_success == False:
  try:
    sql = GeminiProLLM(company_names_sql_prompt, temperature=temperature, topP=.8, topK = 40)
    print(f"SQL: {sql}")

    if starting_value == 1:
      llm_success = RunQuery(sql)
    else:
      llm_success = True
  except:
    # Reduce the temperature for more accurate generation
    temperature = temperature - .05
    print("Regenerating...")

#### Menu Table (Coffee)

In [ ]:
table_name = "company"
field_name = "company_id"
company_ids = GetForeignKeys(dataset_id, table_name, field_name)

table_name = "menu"
primary_key = "menu_id"

schema = GetTableSchema(dataset_id, table_name)

In [ ]:
menu_count = 3 # We mulitple this by 3 to get 9 (one for small, medium and large sizes)

table_name = "menu"
field_name = "item_name"
existing_values = GetDistinctValues(dataset_id, table_name, field_name)

menu_items_prompt = f"""Generate {menu_count} different coffee drink names and return in the below json format.
- The name can be an existing coffee drink or think outside the box for something new.
- The name should be related to coffee.
- Do not use any of these names: [{existing_values}]
- Do not number the results.

JSON format: [ "value" ]
Sample JSON Response: [ "value1", "value2" ]
"""

llm_success = False
temperature=.8
while llm_success == False:
  try:
    menu_names = GeminiProLLM(menu_items_prompt, temperature=temperature, topP=.8, topK = 40)
    menu_names_json = json.loads(menu_names)
    llm_success = True
  except:
    # Reduce the temperature for more accurate generation
    temperature = temperature - .05
    print("Regenerating...")

print(f"menu_names: {menu_names}")

In [ ]:
# Insert data
starting_value = GetStartingValue(dataset_id, table_name, primary_key)

menu_names_sql_prompt=f"""
You are a database engineer and need to generate data for a table for the below schema.
- The schema is for a Google Cloud BigQuery Table.
- The table name is "{project_id}.{dataset_id}.{table_name}".
- Read the description of each field for valid values.
- Do not preface the response with any special characters or 'sql'.
- Generate {menu_count * 3} total rows for this table.
- Valid values for company_id are: {company_ids}
- Valid values for item_name are: {menu_names}
- The starting value of the field {primary_key} is {starting_value}.
- Only generate a single statement, not multiple INSERTs.
- Create a Small, Medium and Large size for each item_name.  The same company_id should be used as well for all 3 sizes.
- For the field "llm_item_image_prompt", limit the text to 256 characters.
- For the field "llm_item_image_url" use the following pattern and replace [[menu_id]] with the generated menu id: https://storage.cloud.google.com/{gcs_storage_bucket}/{gcs_storage_path}[[menu_id]].png

Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

Schema: {schema}
"""

llm_success = False
temperature=.8
while llm_success == False:
  try:
    sql = GeminiProLLM(menu_names_sql_prompt, temperature=temperature, topP=.8, topK = 40)
    print(f"SQL: {sql}")
    llm_success = RunQuery(sql)
  except:
    # Reduce the temperature for more accurate generation
    temperature = temperature - .05
    print("Regenerating...")

In [ ]:
# Query to get a list of menu items and the prompts for the images

sql = f"""SELECT menu_id,
                 item_name,
                 llm_item_image_prompt
          FROM `{project_id}.{dataset_id}.{table_name}`
         WHERE menu_id BETWEEN {starting_value} AND {(starting_value - 1) + (menu_count * 3)}
        ORDER BY menu_id"""

print(f"SQL: {sql}")
df_process = client.query(sql).to_dataframe()
coffee_drink_image_files = []

for row in df_process.itertuples():
  menu_id = row.menu_id
  item_name = row.item_name
  llm_item_image_prompt = row.llm_item_image_prompt + "  The image should be related to a coffee drink you would buy."

  print(f"item_name: {item_name}")
  print(f"llm_item_image_prompt: {llm_item_image_prompt}")
  try:
    image_file = ImageGen(llm_item_image_prompt)
    coffee_drink_image_files.append ({
          "menu_id" : menu_id,
          "item_name" : item_name,
          "llm_item_image_prompt" : llm_item_image_prompt,
          "gcs_storage_bucket" : gcs_storage_bucket,
          "gcs_storage_path" : gcs_storage_path,
          "llm_image_filename" : image_file
      })
  except:
    print("Image failed to generate.")

In [ ]:
# View the results
for item in coffee_drink_image_files:
  print(f"menu_id: {item['menu_id']}")
  print(f"item_name: {item['item_name']}")
  print(f"llm_item_image_prompt: {item['llm_item_image_prompt']}")
  img = Image.open(item["llm_image_filename"])
  img.thumbnail([500,500])
  IPython.display.display(img)

#### Menu Table (Food)

In [ ]:
table_name = "menu"
field_name = "item_name"
existing_values = GetDistinctValues(dataset_id, table_name, field_name)

menu_items_prompt = f"""Generate {menu_count} different foods that you would buy with coffee and return in the below json format.
- The name can be an existing food or think outside the box for something new.
- The items need to be food items and not coffee drinks.
- The name should be related to coffee or a play on words around coffee.
- Do not use any of these names: [{existing_values}]
- Do not number the results.

JSON format: [ "value" ]
Sample JSON Response: [ "value1", "value2" ]
"""

llm_success = False
temperature=.8
while llm_success == False:
  try:
    menu_names = GeminiProLLM(menu_items_prompt, temperature=temperature, topP=.8, topK = 40)
    menu_names_json = json.loads(menu_names)
    llm_success = True
  except:
    # Reduce the temperature for more accurate generation
    temperature = temperature - .05
    print("Regenerating...")

print(f"menu_names: {menu_names}")

In [ ]:
# Insert data
starting_value = GetStartingValue(dataset_id, table_name, primary_key)

menu_names_sql_prompt=f"""
You are a database engineer and need to generate data for a table for the below schema.
- The schema is for a Google Cloud BigQuery Table.
- The table name is "{project_id}.{dataset_id}.{table_name}".
- Read the description of each field for valid values.
- Do not preface the response with any special characters or 'sql'.
- Generate {menu_count} total rows for this table.
- Valid values for company_id are: {company_ids}
- Valid values for item_name are: {menu_names}
- The starting value of the field {primary_key} is {starting_value}.
- Only generate a single statement, not multiple INSERTs.
- Hardcode the field "item_size" to "n/a"
- For the field "llm_item_image_prompt", limit the text to 256 characters.
- For the field "llm_item_image_url" use the following pattern and replace [[menu_id]] with the generated menu id: https://storage.cloud.google.com/{gcs_storage_bucket}/{gcs_storage_path}[[menu_id]].png

Example 1: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Sample'),(2, 'Sample');
Example 2: INSERT INTO `my-dataset.my-dataset.my-table` (field_1, field_2) VALUES (1, 'Data'),(2, 'Data'),(3, 'Data');

Schema: {schema}
"""

llm_success = False
temperature=.8
while llm_success == False:
  try:
    sql = GeminiProLLM(menu_names_sql_prompt, temperature=temperature, topP=.8, topK = 40)
    print(f"SQL: {sql}")
    llm_success = RunQuery(sql)
  except:
    # Reduce the temperature for more accurate generation
    temperature = temperature - .05
    print("Regenerating...")

In [ ]:
# Query to get a list of menu items and the prompts for the images

sql = f"""SELECT menu_id,
                 item_name,
                 llm_item_image_prompt
          FROM `{project_id}.{dataset_id}.{table_name}`
         WHERE menu_id BETWEEN {starting_value} AND { starting_value - 1 + menu_count }
        ORDER BY menu_id"""

print(f"SQL: {sql}")
df_process = client.query(sql).to_dataframe()
food_image_files = []

for row in df_process.itertuples():
  menu_id = row.menu_id
  item_name = row.item_name
  llm_item_image_prompt = row.llm_item_image_prompt

  print(f"item_name: {item_name}")
  print(f"llm_item_image_prompt: {llm_item_image_prompt}")
  try:
    image_file = ImageGen(llm_item_image_prompt)
    food_image_files.append ({
          "menu_id" : menu_id,
          "item_name" : item_name,
          "llm_item_image_prompt" : llm_item_image_prompt,
          "gcs_storage_bucket" : gcs_storage_bucket,
          "gcs_storage_path" : gcs_storage_path,
          "llm_image_filename" : image_file
      })
  except:
    print("Image failed to generate.")

In [ ]:
# View the results
for item in food_image_files:
  print(f"menu_id: {item['menu_id']}")
  print(f"item_name: {item['item_name']}")
  print(f"llm_item_image_prompt: {item['llm_item_image_prompt']}")
  img = Image.open(item["llm_image_filename"])
  img.thumbnail([500,500])
  IPython.display.display(img)

#### Save the results to storage

In [ ]:
# When we create the sample data for our table we also asked for the LLM to generate the correct GCS / HTTP path

# Copy all image files to storage
for item in coffee_drink_image_files:
  copy_file_to_gcs(item["llm_image_filename"],item["gcs_storage_bucket"], item["gcs_storage_path"] + str(item['menu_id']) + ".png")

# Copy all image files to storage
for item in food_image_files:
  copy_file_to_gcs(item["llm_image_filename"],item["gcs_storage_bucket"], item["gcs_storage_path"] + str(item['menu_id']) + ".png")